In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


train_dir = '/content/drive/MyDrive/shoe/train'
test_dir = '/content/drive/MyDrive/shoe/test'
img_size = (224,224)
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

train_data = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_data = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')


def build_vgg16_model(fine_tune_whole=False, fine_tune_partial=False):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

    if fine_tune_whole:
        for layer in base_model.layers:
            layer.trainable = True
    elif fine_tune_partial:
        for layer in base_model.layers[:-4*3]:  # freeze first blocks (example)
            layer.trainable = False
        for layer in base_model.layers[-4*3:]:  # last block trainable
            layer.trainable = True
    else:
        for layer in base_model.layers:
            layer.trainable = False

    x = Flatten()(base_model.output)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(2048, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

model_whole = build_vgg16_model(fine_tune_whole=True)
history_whole = model_whole.fit(train_data, validation_data=test_data, epochs=20)


model_partial = build_vgg16_model(fine_tune_partial=True)
history_partial = model_partial.fit(train_data, validation_data=test_data, epochs=20)


loss_whole, acc_whole = model_whole.evaluate(test_data)
loss_partial, acc_partial = model_partial.evaluate(test_data)
print(f"Whole Fine-Tuning Accuracy: {acc_whole}")
print(f"Partial Fine-Tuning Accuracy: {acc_partial}")